# Lecture 3 -  dates & times, strings,  as well as factors

### Lecture learning objectives:
By the end of this lecture and worksheet 3, students should be able to:

* Manipulate dates and times using the {lubridate} package
* Be able to modify strings in a data frame using regular expressions and the {stringr} package
* Cast categorical columns in a data frame as factors when appropriate, and manipulate factor levels as needed in preparation for data visualisation and statistical analysis (using base R and {forcats} package functions)


In [1]:
library(tidyverse)
library(gapminder)
options(repr.matrix.max.rows = 10)

── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.8     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.1
── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


## Tibbles versus data frames

You have seen and heard me talk about data frames and tibbles in R, and sometimes I carelessly interchange the terms. Let's take moment to discuss the difference between the two!

Tibbles are special data frames, with special/extra properties. The two important ones you will care about are:

- In RStudio, tibbles only output the first 10 rows

- When you numerically subset a data frame to 1 column, you get a vector. However, when you numerically subset a tibble you still get a tibble back.

When you create a data frame using base R functions, either via `data.frame` or one of the base R `read.*` functions, you get an objects whose class is data frame:

In [2]:
example <- data.frame(a = c(1, 5, 9), b = "z", "a", "t")
example

class(example)

a,b,X.a.,X.t.
<dbl>,<chr>,<chr>,<chr>
1,z,a,t
5,z,a,t
9,z,a,t


[1] "data.frame"

Tibbles inherit from the data frame class (meaning that have many of the same properties as data frames), but they also have the extra properties I just discussed:

In [3]:
example2 <- tibble(a = c(1, 5, 9), b = "z", "a", "t")
example

class(example2)

a,b,X.a.,X.t.
<dbl>,<chr>,<chr>,<chr>
1,z,a,t
5,z,a,t
9,z,a,t


[1] "tbl_df"     "tbl"        "data.frame"

Note: there are **some** tidyverse functions that will coerce a data frame to a tibble, because what the user is asking for is not possible with a data frame. One such example is `group_by` (which we will learn about next week):

In [4]:
group_by(example, a) %>% 
    class()

[1] "grouped_df" "tbl_df"     "tbl"        "data.frame"

**Rule of thumb:**  if you want a tibble, it’s on you to know that and express that explicitly with `as_tibble()` (if it’s a data frame to start out with).

## Dates and times

<img src="https://d33wubrfki0l68.cloudfront.net/baa19d0ebf9b97949a7ad259b29a1c4ae031c8e2/8e9b8/diagrams/vectors/summary-tree-s3-1.png" width=300>

*Source: [Advanced R](https://adv-r.hadley.nz/) by Hadley Wickham*

### Working with dates and times

Your weapon: The lubridate package (CRAN; [GitHub](https://github.com/tidyverse/lubridate); [main vignette](https://cran.r-project.org/web/packages/lubridate/vignettes/lubridate.html)).

In [5]:
library(lubridate)


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




### Get your hands on some dates or date-times

Use `lubridate`’s `today` to get today’s date, without any time:

In [6]:
today()

[1] "2022-09-10"

In [7]:
class(today())

[1] "Date"

Use `lubridate`'s `now` to get RIGHT NOW, meaning the date and the time:

In [8]:
now()

[1] "2022-09-10 22:36:51 PDT"

In [9]:
class(now())

[1] "POSIXct" "POSIXt"

### Get date or date-time from character

Use the `lubridate` helpers to convert character or unquoted numbers into dates or date-times:

In [10]:
ymd("2017-01-31")

[1] "2017-01-31"

In [11]:
mdy("January 31st, 2017")

[1] "2017-01-31"

In [12]:
dmy("31-Jan-2017")

[1] "2017-01-31"

In [13]:
ymd(20170131)

[1] "2017-01-31"

In [14]:
ymd_hms("2017-01-31 20:11:59")

[1] "2017-01-31 20:11:59 UTC"

In [15]:
mdy_hm("01/31/2017 08:01")

[1] "2017-01-31 08:01:00 UTC"

You can also force the creation of a date-time from a date by supplying a timezone:

In [16]:
class(ymd(20170131, tz = "UTC"))

[1] "POSIXct" "POSIXt"

### Build date or date-time from parts

Instead of a single string, sometimes you’ll have the individual components of the date-time spread across multiple columns. 

In [17]:
dates <- tibble(year = c(2015, 2016, 2017, 2018, 2019),
               month = c(9, 9, 9, 9, 9),
               day = c(3, 4, 2, 6, 3))

dates

year,month,day
<dbl>,<dbl>,<dbl>
2015,9,3
2016,9,4
2017,9,2
2018,9,6
2019,9,3


To create a date/time from this sort of input, use `make_date` for dates, or `make_datetime` for date-times:

In [18]:
# make a single date from year, month and day
dates %>% 
    mutate(date = make_date(year, month, day))

year,month,day,date
<dbl>,<dbl>,<dbl>,<date>
2015,9,3,2015-09-03
2016,9,4,2016-09-04
2017,9,2,2017-09-02
2018,9,6,2018-09-06
2019,9,3,2019-09-03


### Getting components from a date or date-time

Sometimes you have the date or date-time and you want to extract a component, such as year or day.

In [19]:
datetime <- ymd_hms("2016-07-08 12:34:56")
datetime

[1] "2016-07-08 12:34:56 UTC"

In [20]:
year(datetime)

[1] 2016

In [21]:
month(datetime)

[1] 7

In [22]:
mday(datetime)

[1] 8

In [23]:
yday(datetime)

[1] 190

In [24]:
wday(datetime, label = TRUE, abbr = FALSE)

[1] Friday
7 Levels: Sunday < Monday < Tuesday < Wednesday < Thursday < ... < Saturday

For `month` and `wday` you can set `label = TRUE` to return the abbreviated name of the month or day of the week. Set `abbr = FALSE` to return the full name.

### More date and time wrangling possibilities:

- [Tools for working with time spans](https://r4ds.had.co.nz/dates-and-times.html#time-spans), to calculate durations, periods and intervals
- [Tools for dealing with time zones](https://r4ds.had.co.nz/dates-and-times.html#time-zones)

## String manipulations

#### The recommended tools:

- **[stringr](https://stringr.tidyverse.org/) package:** A core package in the tidyverse. Main functions start with str_. Auto-complete is your friend. [`stringr` cheatsheet](https://github.com/rstudio/cheatsheets/blob/master/strings.pdf)

- **[tidyr](https://tidyr.tidyverse.org/) package:** Useful for functions that split one character vector into many and vice versa: `separate`, `unite`, `extract`. [`tidyr` cheatsheet](https://github.com/rstudio/cheatsheets/blob/master/data-import.pdf)

- **Base functions:** `nchar`, `strsplit`, `substr`, `paste`, and `paste0`.

#### Regex-free string manipulation with stringr and tidyr

Basic string manipulation tasks:

- Study a single character vector
    - How long are the strings?
    - Presence/absence of a literal string
- Operate on a single character vector
    - Keep/discard elements that contain a literal string
    - Split into two or more character vectors using a fixed delimiter
    - Snip out pieces of the strings based on character position
    - Collapse into a single string
- Operate on two or more character vectors
    - Glue them together element-wise to get a new character vector.

`fruit`, `words`, and `sentences` are character vectors that ship with `stringr` for practicing.

NOTE - we will be working with vectors today. If you want to operate on data frames, you will need to use these functions inside of data frame/tibble functions, like `filter` and `mutate`.

#### Detect or filter on a target string

Determine presence/absence of a literal string with `str_detect`. Spoiler: later we see `str_detect` also detects regular expressions.

Which fruits actually use the word “fruit”?

In [25]:
# detect "fruit"
#typeof(fruit)
#fruit
str_detect(fruit, "fruit")

[1] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE
[13] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[25] FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE
[37] FALSE FALSE  TRUE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE
[49] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE
[61] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[73] FALSE FALSE  TRUE FALSE FALSE FALSE  TRUE FALSE

What’s the easiest way to get the actual fruits that match? Use `str_subset` to keep only the matching elements. Note we are storing this new vector `my_fruit` to use in later examples!

In [26]:
# subset "fruit"
my_fruit <- str_subset(fruit, "fruit")
my_fruit

[1] "breadfruit"   "dragonfruit"  "grapefruit"   "jackfruit"    "kiwi fruit"  
[6] "passionfruit" "star fruit"   "ugli fruit"

#### String splitting by delimiter

Use `stringr::str_split` to split strings on a delimiter. 

Some of our fruits are compound words, like “grapefruit”, but some have two words, like “ugli fruit”. Here we split on a single space " ", but show use of a regular expression later.

In [27]:
# split on " "
str_split(my_fruit, " ")

[[1]]
[1] "breadfruit"

[[2]]
[1] "dragonfruit"

[[3]]
[1] "grapefruit"

[[4]]
[1] "jackfruit"

[[5]]
[1] "kiwi"  "fruit"

[[6]]
[1] "passionfruit"

[[7]]
[1] "star"  "fruit"

[[8]]
[1] "ugli"  "fruit"

It’s bummer that we get a list back. But it must be so! In full generality, split strings must return list, because who knows how many pieces there will be?

If you are willing to commit to the number of pieces, you can use `str_split_fixed` and get a character matrix. You’re welcome!

In [28]:
str_split_fixed(my_fruit, pattern = " ", n = 2)

breadfruit,
dragonfruit,
grapefruit,
jackfruit,
kiwi,fruit
passionfruit,
star,fruit
ugli,fruit


If the to-be-split variable lives in a data frame, `tidyr::separate` will split it into 2 or more variables:

In [29]:
# separate on " "
my_fruit[5] <- "yellow kiwi fruit"
my_fruit
tibble(unsplit = my_fruit) %>% 
    separate(unsplit, into = c("pre", "post"), sep = " ")

[1] "breadfruit"        "dragonfruit"       "grapefruit"       
[4] "jackfruit"         "yellow kiwi fruit" "passionfruit"     
[7] "star fruit"        "ugli fruit"

Warning message:
“Expected 2 pieces. Additional pieces discarded in 1 rows [5].”
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 5 rows [1, 2, 3, 4, 6].”


pre,post
<chr>,<chr>
breadfruit,NA
dragonfruit,NA
grapefruit,NA
jackfruit,NA
yellow,kiwi
passionfruit,NA
star,fruit
ugli,fruit


#### Substring extraction (and replacement) by position

Count characters in your strings with `str_length`. *Note this is different from the length of the character vector itself.*

In [30]:
# get length of each string
str_length(my_fruit)
my_fruit

[1] 10 11 10  9 17 12 10 10

[1] "breadfruit"        "dragonfruit"       "grapefruit"       
[4] "jackfruit"         "yellow kiwi fruit" "passionfruit"     
[7] "star fruit"        "ugli fruit"

You can snip out substrings based on character position with `str_sub`.

In [31]:
# remove first three strings
str_sub(my_fruit, 1, 3)

[1] "bre" "dra" "gra" "jac" "yel" "pas" "sta" "ugl"

Finally, `str_sub` also works for assignment, i.e. on the left hand side of `<-`

In [32]:
# replace three characters with AAA
str_sub(my_fruit, 1, 3)  <- "AAA"
my_fruit

[1] "AAAadfruit"        "AAAgonfruit"       "AAApefruit"       
[4] "AAAkfruit"         "AAAlow kiwi fruit" "AAAsionfruit"     
[7] "AAAr fruit"        "AAAi fruit"

#### Collapse a vector

You can collapse a character vector of length n > 1 to a single string with `str_c`, which also has other uses (see the next section).

In [33]:
# collapse a character vector into one 
head(fruit) %>% 
    str_c(collapse = "-")

[1] "apple-apricot-avocado-banana-bell pepper-bilberry"

#### Create a character vector by catenating multiple vectors

If you have two or more character vectors of the same length, you can glue them together element-wise, to get a new vector of that length. Here are some … awful smoothie flavors?

In [34]:
# concatenate character vectors
fruit[1:4]
fruit[5:8]
str_c(fruit[1:4], fruit[5:8], sep = "")

[1] "apple"   "apricot" "avocado" "banana"

[1] "bell pepper"  "bilberry"     "blackberry"   "blackcurrant"

[1] "applebell pepper"   "apricotbilberry"    "avocadoblackberry" 
[4] "bananablackcurrant"

If the to-be-combined vectors are variables in a data frame, you can use `tidyr::unite` to make a single new variable from them.

In [35]:
# concatenate character vectors when they are in a data frame
tibble(fruit1 = fruit[1:4],
      fruit2 = fruit[5:8]) %>% 
    unite("flavour_combo", fruit1, fruit2, sep = " & ")

flavour_combo
<chr>
apple & bell pepper
apricot & bilberry
avocado & blackberry
banana & blackcurrant


####  Substring replacement

You can replace a pattern with `str_replace`. Here we use an explicit string-to-replace, but later we revisit with a regular expression.

In [36]:
# replace fruit with vegetable
my_fruit <- str_subset(fruit, "fruit")
my_fruit
str_replace(my_fruit, "fruit", "vegetable")

[1] "breadfruit"   "dragonfruit"  "grapefruit"   "jackfruit"    "kiwi fruit"  
[6] "passionfruit" "star fruit"   "ugli fruit"

[1] "breadvegetable"   "dragonvegetable"  "grapevegetable"   "jackvegetable"   
[5] "kiwi vegetable"   "passionvegetable" "star vegetable"   "ugli vegetable"

- A special case that comes up a lot is replacing `NA`, for which there is `str_replace_na`.

- If the NA-afflicted variable lives in a data frame, you can use `tidyr::replace_na`.

#### Other `str_*` functions?

There are many many other useful `str_*` functions from the `stringr` package. Too many to go through them all here. If these shown in lecture aren't what you need, then you should try `?str` + tab to see the possibilities:

In [37]:
?str_

No documentation for ‘str_’ in specified packages and libraries:
you could try ‘??str_’

#### Regular expressions with stringr

or...

![](https://stat545.com/img/regexbytrialanderror-big-smaller.png)

#### Examples with gapminder

In [38]:
library(gapminder)
head(gapminder)

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Afghanistan,Asia,1952,28.801,8425333,779.4453
Afghanistan,Asia,1957,30.332,9240934,820.8530
Afghanistan,Asia,1962,31.997,10267083,853.1007
Afghanistan,Asia,1967,34.020,11537966,836.1971
Afghanistan,Asia,1972,36.088,13079460,739.9811
Afghanistan,Asia,1977,38.438,14880372,786.1134


#### Filtering rows with `str_detect`

Let's filter for rows where the country name starts with "AL":

In [39]:
library(tidyverse)
library(gapminder)

In [40]:
# detect countries that start with "AL"
gapminder %>% 
    filter(str_detect(country, "^Al")) %>% 
    pull(country) %>% 
    unique() %>% 
    length()

[1] 2

And now rows where the country ends in `tan`:

In [41]:
# detect countries that end with "tan"
gapminder %>% 
    filter(str_detect(country, "tan$"))

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
Afghanistan,Asia,1952,28.801,8425333,779.4453
Afghanistan,Asia,1957,30.332,9240934,820.8530
Afghanistan,Asia,1962,31.997,10267083,853.1007
Afghanistan,Asia,1967,34.020,11537966,836.1971
Afghanistan,Asia,1972,36.088,13079460,739.9811
⋮,⋮,⋮,⋮,⋮,⋮
Pakistan,Asia,1987,58.245,105186881,1704.687
Pakistan,Asia,1992,60.838,120065004,1971.829
Pakistan,Asia,1997,61.818,135564834,2049.351


Or countries containing ", Dem. Rep." :

In [42]:
# detect countries that contain ", Dem. Rep." 
gapminder %>% 
    filter(str_detect(country, "\\, Dem. Rep."))

country,continent,year,lifeExp,pop,gdpPercap
<fct>,<fct>,<int>,<dbl>,<int>,<dbl>
"Congo, Dem. Rep.",Africa,1952,39.143,14100005,780.5423
"Congo, Dem. Rep.",Africa,1957,40.652,15577932,905.8602
"Congo, Dem. Rep.",Africa,1962,42.122,17486434,896.3146
"Congo, Dem. Rep.",Africa,1967,44.056,19941073,861.5932
"Congo, Dem. Rep.",Africa,1972,45.989,23007669,904.8961
⋮,⋮,⋮,⋮,⋮,⋮
"Korea, Dem. Rep.",Asia,1987,70.647,19067554,4106.492
"Korea, Dem. Rep.",Asia,1992,69.978,20711375,3726.064
"Korea, Dem. Rep.",Asia,1997,67.727,21585105,1690.757


Replace ", Dem. Rep." with " Democratic Republic":

In [43]:
# replace ", Dem. Rep." with " Democratic Republic"
gapminder %>% 
    mutate(country = str_replace(country,
                                "\\, Dem. Rep.",
                                " Democratic Republic")) %>%
    filter(country == "Korea Democratic Republic")

country,continent,year,lifeExp,pop,gdpPercap
<chr>,<fct>,<int>,<dbl>,<int>,<dbl>
Korea Democratic Republic,Asia,1952,50.056,8865488,1088.278
Korea Democratic Republic,Asia,1957,54.081,9411381,1571.135
Korea Democratic Republic,Asia,1962,56.656,10917494,1621.694
Korea Democratic Republic,Asia,1967,59.942,12617009,2143.541
Korea Democratic Republic,Asia,1972,63.983,14781241,3701.622
⋮,⋮,⋮,⋮,⋮,⋮
Korea Democratic Republic,Asia,1987,70.647,19067554,4106.492
Korea Democratic Republic,Asia,1992,69.978,20711375,3726.064
Korea Democratic Republic,Asia,1997,67.727,21585105,1690.757


### Extract matches (optional)

To extract the actual text of a match, use `str_extract`. 

We’re going to need a more complicated example, the Harvard `sentences` from the `stringr` package: 

In [44]:
typeof(sentences)
head(sentences)

[1] "character"

[1] "The birch canoe slid on the smooth planks." 
[2] "Glue the sheet to the dark blue background."
[3] "It's easy to tell the depth of a well."     
[4] "These days a chicken leg is a rare dish."   
[5] "Rice is often served in round bowls."       
[6] "The juice of lemons makes fine punch."

Say we want to extract all of the colours used in the sentences. We can do this by creating a pattern which would match them, and passing that and our vector of sentences to `str_extract`:

In [45]:
colours <- "red|orange|yellow|green|blue|purple"

In [46]:
# extract colours used in sentences
str_extract(sentences, colours)

[1] NA       "blue"   NA       NA       NA       NA       NA       NA      
  [9] NA       NA       NA       NA       NA       NA       NA       NA      
 [17] NA       NA       NA       NA       NA       NA       NA       NA      
 [25] NA       "blue"   NA       "red"    NA       NA       NA       NA      
 [33] NA       NA       NA       NA       NA       NA       NA       NA      
 [41] NA       NA       NA       "red"    NA       NA       NA       NA      
 [49] NA       NA       NA       NA       NA       NA       NA       NA      
 [57] NA       NA       NA       NA       NA       NA       NA       NA      
 [65] NA       NA       NA       NA       NA       NA       NA       NA      
 [73] NA       NA       NA       NA       NA       NA       NA       NA      
 [81] NA       "red"    NA       NA       NA       NA       NA       NA      
 [89] NA       NA       NA       "blue"   NA       NA       NA       NA      
 [97] NA       NA       NA       NA       NA       NA       NA       NA      
[105] NA       NA       NA       NA       NA       NA       NA       "yellow"
[113] NA       NA       NA       "red"    NA       NA       NA       NA      
[121] NA       NA       NA       NA       NA       NA       NA       NA      
[129] NA       NA       NA       NA       NA       NA       NA       NA      
[137] NA       NA       NA       NA       NA       NA       NA       NA      
[145] NA       "red"    NA       "green"  "red"    NA       NA       NA      
[153] NA       NA       NA       NA       NA       NA       NA       "red"   
[161] NA       NA       NA       NA       NA       NA       NA       NA      
[169] NA       NA       NA       NA       NA       "blue"   "red"    NA      
[177] "red"    "red"    NA       NA       NA       NA       NA       "red"   
[185] NA       NA       NA       NA       NA       NA       NA       NA      
[193] NA       NA       NA       NA       NA       NA       NA       NA      
[201] NA       NA       NA       "blue"   NA       NA       NA       NA      
[209] NA       NA       NA       NA       NA       NA       "red"    NA      
[217] "blue"   NA       NA       "red"    NA       NA       NA       "green" 
[225] NA       NA       NA       NA       NA       NA       NA       NA      
[233] NA       NA       NA       NA       NA       NA       NA       NA      
[241] NA       NA       NA       NA       NA       NA       "red"    NA      
[249] NA       NA       NA       NA       NA       NA       "red"    "red"   
[257] NA       NA       NA       NA       NA       NA       NA       NA      
[265] NA       NA       NA       NA       NA       NA       NA       NA      
[273] NA       "red"    NA       NA       "red"    NA       "red"    NA      
[281] NA       NA       NA       NA       NA       NA       NA       "green" 
[289] NA       NA       NA       NA       "red"    NA       NA       NA      
[297] NA       NA       NA       NA       NA       "green"  NA       NA      
[305] NA       NA       NA       NA       NA       NA       "red"    NA      
[313] NA       NA       NA       NA       NA       NA       "purple" NA      
[321] NA       NA       NA       NA       NA       NA       NA       NA      
[329] NA       "green"  NA       NA       NA       NA       NA       NA      
[337] NA       NA       NA       NA       NA       NA       NA       NA      
[345] "red"    NA       NA       NA       NA       NA       NA       NA      
[353] NA       NA       NA       NA       NA       NA       NA       NA      
[361] NA       NA       NA       NA       NA       NA       NA       "red"   
[369] NA       NA       NA       "red"    NA       NA       NA       NA      
[377] NA       NA       NA       NA       NA       NA       NA       NA      
[385] NA       NA       "red"    "red"    NA       NA       NA       NA      
[393] NA       NA       NA       NA       NA       NA       NA       NA      
[401] NA       NA       NA       NA       NA       NA       NA       NA      
[409] NA       NA       

`str_extract` only returns the first match for each element. To return all matches from an element we need to use `str_extract_all`:

In [47]:
# extract all colours used in sentences
str_extract_all(sentences, colours)

[[1]]
character(0)

[[2]]
[1] "blue"

[[3]]
character(0)

[[4]]
character(0)

[[5]]
character(0)

[[6]]
character(0)

[[7]]
character(0)

[[8]]
character(0)

[[9]]
character(0)

[[10]]
character(0)

[[11]]
character(0)

[[12]]
character(0)

[[13]]
character(0)

[[14]]
character(0)

[[15]]
character(0)

[[16]]
character(0)

[[17]]
character(0)

[[18]]
character(0)

[[19]]
character(0)

[[20]]
character(0)

[[21]]
character(0)

[[22]]
character(0)

[[23]]
character(0)

[[24]]
character(0)

[[25]]
character(0)

[[26]]
[1] "blue"

[[27]]
character(0)

[[28]]
[1] "red"

[[29]]
character(0)

[[30]]
character(0)

[[31]]
character(0)

[[32]]
character(0)

[[33]]
character(0)

[[34]]
character(0)

[[35]]
character(0)

[[36]]
character(0)

[[37]]
character(0)

[[38]]
character(0)

[[39]]
character(0)

[[40]]
character(0)

[[41]]
character(0)

[[42]]
character(0)

[[43]]
character(0)

[[44]]
[1] "red"

[[45]]
character(0)

[[46]]
character(0)

[[47]]
character(0)

[[48]]
character(0)

[[49]]
character(0)

[[50]]
character(0)

[[51]]
character(0)

[[52]]
character(0)

[[53]]
character(0)

[[54]]
character(0)

[[55]]
character(0)

[[56]]
character(0)

[[57]]
character(0)

[[58]]
character(0)

[[59]]
character(0)

[[60]]
character(0)

[[61]]
character(0)

[[62]]
character(0)

[[63]]
character(0)

[[64]]
character(0)

[[65]]
character(0)

[[66]]
character(0)

[[67]]
character(0)

[[68]]
character(0)

[[69]]
character(0)

[[70]]
character(0)

[[71]]
character(0)

[[72]]
character(0)

[[73]]
character(0)

[[74]]
character(0)

[[75]]
character(0)

[[76]]
character(0)

[[77]]
character(0)

[[78]]
character(0)

[[79]]
character(0)

[[80]]
character(0)

[[81]]
character(0)

[[82]]
[1] "red"

[[83]]
character(0)

[[84]]
character(0)

[[85]]
character(0)

[[86]]
character(0)

[[87]]
character(0)

[[88]]
character(0)

[[89]]
character(0)

[[90]]
character(0)

[[91]]
character(0)

[[92]]
[1] "blue"

[[93]]
character(0)

[[94]]
character(0)

[[95]]
character(0)

[[96]]
character(0)

[[97]]
character(0)

[[98]]
character(0)

[[99]]
character(0)

[[100]]
character(0)

[[101]]
character(0)

[[102]]
character(0)

[[103]]
character(0)

[[104]]
character(0)

[[105]]
character(0)

[[106]]
character(0)

[[107]]
character(0)

[[108]]
character(0)

[[109]]
character(0)

[[110]]
character(0)

[[111]]
character(0)

[[112]]
[1] "yellow"

[[113]]
character(0)

[[114]]
character(0)

[[115]]
character(0)

[[116]]
[1] "red"

[[117]]
character(0)

[[118]]
character(0)

[[119]]
character(0)

[[120]]
character(0)

[[121]]
character(0)

[[122]]
character(0)

[[123]]
character(0)

[[124]]
character(0)

[[125]]
character(0)

[[126]]
character(0)

[[127]]
character(0)

[[128]]
character(0)

[[129]]
character(0)

[[130]]
character(0)

[[131]]
character(0)

[[132]]
character(0)

[[133]]
character(0)

[[134]]
character(0)

[[135]]
character(0)

[[136]]
character(0)

[[137]]
character(0)

[[138]]
character(0)

[[139]]
character(0)

[[140]]
character(0)

[[141]]
character(0)

[[142]]
character(0)

[[143]]
character(0)

[[144]]
character(0)

[[145]]
character(0)

[[146]]
[1] "red"

[[147]]
character(0)

[[148]]
[1] "green"

[[149]]
[1] "red"

[[150]]
character(0)

[[151]]
character(0)

[[152]]
character(0)

[[153]]
character(0)

[[154]]
character(0)

[[155]]
character(0)

[[156]]
character(0)

[[157]]
character(0)

[[158]]
character(0)

[[159]]
character(0)

[[160]]
[1] "red"

[[161]]
character(0)

[[162]]
character(0)

[[163]]
character(0)

[[164]]
character(0)

[[165]]
character(0)

[[166]]
character(0)

[[167]]
character(0)

[[168]]
character(0)

[[169]]
character(0)

[[170]]
character(0)

[[171]]
character(0)

[[172]]
character(0)

[[173]]
character(0)

[[174]]
[1] "blue"

[[175]]
[1] "red"

[[176]]
character(0)

[[177]]
[1] "red"

[[178]]
[1] "red"

[[179]]
character(0)

[[180]]
character(0)

[[181]]
character(0)

[[182]]
character(0)

[[183]]
character(0)

[[184]]
[1] "red"

[[185]]
character(0)

[[186]]
character(0)

[[187]]
character(0)

[[188]]
character(0)

[[189]]
charac

Note: `str_extract` returns a character vector, whereas `str_extract_all` returns a litst. This is because when asking for multiple matches back, you do not know how many you will get, and thus we cannot expect a rectangular shape.

### Capture groups (optional)

You can also use parentheses to extract parts of a complex match. 

For example, imagine we want to extract nouns from the sentences. As a heuristic, we’ll look for any word that comes after “a” or “the”. Defining a “word” in a regular expression is a little tricky, so here I use a simple approximation: a sequence of at least one character that isn’t a space.

In [48]:
# extract nouns from sentences
noun <- "(a|the) ([^ ]+)"

str_match(sentences, noun) %>% 
    head()

the smooth,the,smooth
the sheet,the,sheet
the depth,the,depth
a chicken,a,chicken
NA,NA,NA
NA,NA,NA


Like `str_extract`, if you want all matches for each string, you’ll need `str_match_all`

### Summary of string manipulation functions covered so far:

| function | description |
|----------|-------------|
| `str_detect` | Detects elements in a vector that match a pattern, returns a vector of logicals |
| `srt_subset` | Detects and returns elements in a vector that match a pattern |
| `str_split` | Split strings in a vector on a delimiter. Returns a list (used `str_split_fixed` to get a matrix) |
| `separate` | Split character vectors from a data frame on a delimiter which get returned as additional columns in the data frame |
| `str_length` | Counts the number of characters for each element of a character vector, and returns a numeric vector of the counts |
| `str_sub` | Remove substrings based on character position |
| `str_c` | Collapse and/or concatenate elements from a character vector(s) |
| `unite` | Concatenate elements from character vectors from a data frame to create a single column |
| `str_replace` | Replace a pattern in a vector of character vectors with a given string |
| `str_extract` | Extract the actual text of a match from a character vector |
| `str_match` | Use capture groups to extract parts of a complex match from a character vector, returns the match and the capture groups as columns of a matrix |

## Factors

<img src="https://d33wubrfki0l68.cloudfront.net/baa19d0ebf9b97949a7ad259b29a1c4ae031c8e2/8e9b8/diagrams/vectors/summary-tree-s3-1.png" width=300>

*Source: [Advanced R](https://adv-r.hadley.nz/) by Hadley Wickham*

### Be the boss of your factors

- I love and hate factors

- I love them for data visualization and statistics because I do not need to make dummy variables 

- I hate them because if you are not careful, they fool you because they look like character vectors. And when you treat them like character vectors you get cryptic error messages, like we saw when we tried to do a conditional mutate on the gapminder data set

#### Tidyverse philosophy for factors

- Humans, not computers should decide which columns are factors

- Factors are not that useful until you are at the end of your data wrangling, before that you want character vectors so you can do string manipulations

- Tidyverse functions, like `tibble`, and `read_csv` give you columns with strings as character vectors, Base R functions like `data.frame` and `read.csv`

### Factor inspection

Get to know your factor before you start touching it! It’s polite. Let’s use `gapminder$continent` as our example.

In [49]:
str(gapminder$continent)

 Factor w/ 5 levels "Africa","Americas",..: 3 3 3 3 3 3 3 3 3 3 ...


In [50]:
levels(gapminder$continent)

[1] "Africa"   "Americas" "Asia"     "Europe"   "Oceania"

In [51]:
nlevels(gapminder$continent)

[1] 5

In [52]:
class(gapminder$continent)

[1] "factor"

### Dropping unused levels

Just because you drop all the rows corresponding to a specific factor level, the levels of the factor itself do not change. Sometimes all these unused levels can come back to haunt you later, e.g., in figure legends.

Watch what happens to the levels of `country` when we filter Gapminder to a handful of countries:

In [53]:
nlevels(gapminder$country)

[1] 142

In [54]:
h_countries <- gapminder %>% 
    filter(country %in% c("Egypt", "Haiti", "Romania", "Thailand", "Venezuela"))

nlevels(h_countries$country)

[1] 142

huh? Even though `h_gap` only has data for a handful of countries, we are still schlepping around all 142 levels from the original `gapminder` tibble.

How to get rid of them? We'll use the `forcats::fct_drop` function to do this:

In [55]:
h_countries$country %>% nlevels()

[1] 142

In [56]:
h_countries$country %>% 
    fct_drop() %>% 
    nlevels

[1] 5

### Change order of the levels, principled

By default, factor levels are ordered alphabetically. Which might as well be random, when you think about it! It is preferable to order the levels according to some principle:

- Frequency. Make the most common level the first and so on.
- Another variable. Order factor levels according to a summary statistic for another variable. Example: order Gapminder countries by life expectancy.

First, let’s order continent by frequency, forwards and backwards. This is often a great idea for tables and figures, esp. frequency barplots.

In [57]:
## default order is alphabetical
gapminder$continent %>% 
    levels()

[1] "Africa"   "Americas" "Asia"     "Europe"   "Oceania"

Let's use `forcats::fct_infreq` to order by frequency:

In [58]:
gapminder$continent %>% 
    fct_infreq() %>% 
    levels()

gap2 <- gapminder %>% 
    mutate(continent = fct_infreq(continent))

gap2$continent %>% 
    levels()

[1] "Africa"   "Asia"     "Europe"   "Americas" "Oceania"

[1] "Africa"   "Asia"     "Europe"   "Americas" "Oceania"

Or reverse frequency:

In [59]:
gapminder$continent %>% 
  fct_infreq() %>%
  fct_rev() %>% 
  levels()

[1] "Oceania"  "Americas" "Europe"   "Asia"     "Africa"

### Order one variable by another

You can use `forcats::fct_reorder` to order one variable by another.

The factor is the grouping variable and the default summarizing function is `median` but you can specify something else.

In [60]:
## order countries by median life expectancy
fct_reorder(gapminder$country, gapminder$lifeExp) %>% 
    levels() %>% 
    head()

[1] "Sierra Leone"  "Guinea-Bissau" "Afghanistan"   "Angola"       
[5] "Somalia"       "Guinea"

Using `min` instead to reorder the factors:

In [61]:
## order accoring to minimum life exp instead of median
fct_reorder(gapminder$country, gapminder$lifeExp, min) %>% 
    levels() %>% 
    head()

[1] "Rwanda"       "Afghanistan"  "Gambia"       "Angola"       "Sierra Leone"
[6] "Cambodia"

### Change order of the levels, “because I said so”

Sometimes you just want to hoist one or more levels to the front. Why? Because I said so (sometimes really useful when creating visualizations).

Reminding ourselves of the level order for `gapminder$continent`:

In [62]:
gapminder$continent %>% levels()

[1] "Africa"   "Americas" "Asia"     "Europe"   "Oceania"

Reorder and put Asia and Africa first:

In [63]:
gapminder$continent %>% 
    fct_relevel("Asia", "Africa")  %>% 
    levels()

[1] "Asia"     "Africa"   "Americas" "Europe"   "Oceania"

#### Why do we need to know how to do this?

- Factor levels impact statistical analysis & data visualization!

- For example, these two barcharts of frequency by continent differ only in the order of the continents. Which do you prefer? Discuss with your neighbour.

![](img/factor_viz.png)

## What did we learn today?

- The differences between data frames and tibbles

- The beautiful {lubridate} package for working with dates and times

- Tools for manipulating and working with character data in the {stringr} and {tidyr} packages

- How to take control of our factors using {forcats} and how to investigate factors using base R functions

## Attributions
- [Stat 545](https://stat545.com/) created by Jenny Bryan
- [R for Data Science](https://r4ds.had.co.nz/index.html) by Garrett Grolemund & Hadley Wickham